# The Purpose Of this Experiment is to Turn Off Generator in Case 9 Bus IEEE by Matpower, *Simulation Use Optimal Power Flow not PF or DCPF*.

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Base Case Without Turn Off Generator Status

In [ ]:
from matpower import start_instance
from matpowercaseframes import CaseFrames

m = start_instance()

path = "../data/case9.m"
cf = CaseFrames(path)

# TRY USING ACOPF
mpc = cf.to_mpc()
mpopt = m.mpoption('verbose', 2)
[baseMVA, bus, gen, gencost, branch, f, success, et] = m.runopf(mpc, mpopt, nout='max_nout')


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0      8363.125                   0.155       0.1765      57.1483            0
  1     5307.5519    0.68632   0.00900309      8.30505      15.6381     0.166397
  2     5391.0949    0.24288  0.000359195     0.630698      2.20652   0.00545763
  3     5352.5722   0.088637   3.7341e-05     0.141404     0.216699   0.00250292
  4     5315.4636    0.17189  0.000106315    0.0190754      0.02128    0.0024171
  5     5312.3257   0.066353  0.000123278   0.00135236   0.00351135  0.000204886
  6     5301.3809    0.19111   0.00120494   0.00174642  0.000962847  0.000714764
  7     5296.7947    0.11693  0.000322236  7.55989e-05  0.000161495  0.000299729
  8     5

## Case 1: Edit Generator Status Via MatpowerCaseframes = before simulation has intention to turn off one generator status.

In [ ]:
from matpower import start_instance
from matpowercaseframes import CaseFrames

In [ ]:
m = start_instance()

In [ ]:
# Ensure case path is from matpower
from matpower import path_matpower_cases

file_path = f"{path_matpower_cases}/case9.m"

with open(file_path, 'r') as file:
    content = file.read()

print(content)

function mpc = case9
%CASE9    Power flow data for 9 bus, 3 generator case.
%   Please see CASEFORMAT for details on the case file format.
%
%   Based on data from p. 70 of:
%
%   Chow, J. H., editor. Time-Scale Modeling of Dynamic Networks with
%   Applications to Power Systems. Springer-Verlag, 1982.
%   Part of the Lecture Notes in Control and Information Sciences book
%   series (LNCIS, volume 46)
%
%   which in turn appears to come from:
%
%   R.P. Schulz, A.E. Turner and D.N. Ewart, "Long Term Power System
%   Dynamics," EPRI Report 90-7-0, Palo Alto, California, 1974.

%   MATPOWER

%% MATPOWER Case Format : Version 2
mpc.version = '2';

%%-----  Power Flow Data  -----%%
%% system MVA base
mpc.baseMVA = 100;

%% bus data
%	bus_i	type	Pd	Qd	Gs	Bs	area	Vm	Va	baseKV	zone	Vmax	Vmin
mpc.bus = [
	1	3	0	0	0	0	1	1	0	345	1	1.1	0.9;
	2	2	0	0	0	0	1	1	0	345	1	1.1	0.9;
	3	2	0	0	0	0	1	1	0	345	1	1.1	0.9;
	4	1	0	0	0	0	1	1	0	345	1	1.1	0.9;
	5	1	90	30	0	0	1	1	0	345	1	1.1	0.9;
	6	1	0	0	0	0	1	1	0	3

In [ ]:
path = "../data/case9.m"
cf = CaseFrames(path)

In [ ]:
# Check Attributes
cf.attributes

['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost']

In [ ]:
cf.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,72.3,27.03,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,163.0,6.54,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,85.0,-10.95,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Check wether there is Generator Status or not
cf.gen.columns

Index(['GEN_BUS', 'PG', 'QG', 'QMAX', 'QMIN', 'VG', 'MBASE', 'GEN_STATUS',
       'PMAX', 'PMIN', 'PC1', 'PC2', 'QC1MIN', 'QC1MAX', 'QC2MIN', 'QC2MAX',
       'RAMP_AGC', 'RAMP_10', 'RAMP_30', 'RAMP_Q', 'APF'],
      dtype='object')

In [ ]:
cf.gen['PG']

1     72.3
2    163.0
3     85.0
Name: PG, dtype: float64

In [ ]:
cf.gen['GEN_STATUS']

1    1.0
2    1.0
3    1.0
Name: GEN_STATUS, dtype: float64

## Try to turn off generator status at gen 2

In [ ]:
# Because index in Python is from 0 then Gen Status columns must be in index 7
# This code is just for checking the truth
import pandas as pd

# cols copied from cf.gen.columns
cols = pd.Index([
    'GEN_BUS', 'PG', 'QG', 'QMAX', 'QMIN', 'VG', 'MBASE', 'GEN_STATUS',
    'PMAX', 'PMIN', 'PC1', 'PC2', 'QC1MIN', 'QC1MAX', 'QC2MIN', 'QC2MAX',
    'RAMP_AGC', 'RAMP_10', 'RAMP_30', 'RAMP_Q', 'APF'
])

# cari posisi GEN_STATUS
idx = cols.get_loc("GEN_STATUS")
print(idx)


7


In [ ]:
cf.gen.iloc[0:3,7]

1    1.0
2    1.0
3    1.0
Name: GEN_STATUS, dtype: float64

In [ ]:
# If we want to know exact value must understand index Python, because gen and gen status have 3 then row index are 0,1,2
# Hence if we want to take gen status value from gen 2, 
# we must write index 1 (generator 2) row and index 7 columns from caseframes
cf.gen.iloc[1,7]

np.float64(1.0)

In [ ]:
# Change gen status value of gen 2 to 0 (zero)
cf.gen.iloc[1,7] = 0

In [ ]:
# Check
cf.gen['GEN_STATUS']

1    1.0
2    0.0
3    1.0
Name: GEN_STATUS, dtype: float64

In [ ]:
# TRY USING ACOPF
mpc = cf.to_mpc()
mpopt = m.mpoption('verbose', 2)
[baseMVA, bus, gen, gencost, branch, f, success, et] = m.runopf(mpc, mpopt, nout='max_nout')


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0          5535                    0.14       0.1765      57.0117            0
  1     7197.6704    0.52932    0.0149403      7.34106      11.3047     0.107027
  2     7373.1882    0.42799   0.00136366     0.755348        2.122    0.0102059
  3     7371.1188   0.021125  1.87368e-05    0.0689365     0.222036  0.000119116
  4     7366.8124   0.043537  3.30514e-05   0.00636179    0.0224058  0.000247908
  5     7360.7088   0.072788  0.000218749  0.000746286   0.00422756  0.000351449
  6      7341.446    0.23039   0.00171759   0.00268994   0.00111216   0.00110956
  7      7335.251    0.11765  0.000371412  0.000212541  0.000179507  0.000357238
  8     7

# Case 2: Edit Generator Status Via Matpower Style (Matrix Numpy in Matpower case (mpc) structure), *Simulation Using Optimal Power Flow and Using Eval*.

In [ ]:
# Base Case Running Simulation

from matpower import start_instance

m = start_instance()

m.eval(
    """
    mpopt = mpoption('verbose', 2);
    mpc = loadcase('case9');
    _r1 = runopf(mpc, mpopt);
    """
)

# fech data to python (.eval is used because .pull is not working in acessing field)
r1_mpc = m.eval(
    "struct("
    " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus, 'gen', _r1.gen,"
    " 'branch', _r1.branch, 'gencost', _r1.gencost);"
)


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0      8363.125                   0.155       0.1765      57.1483            0
  1     5307.5519    0.68632   0.00900309      8.30505      15.6381     0.166397
  2     5391.0949    0.24288  0.000359195     0.630698      2.20652   0.00545763
  3     5352.5722   0.088637   3.7341e-05     0.141404     0.216699   0.00250292
  4     5315.4636    0.17189  0.000106315    0.0190754      0.02128    0.0024171
  5     5312.3257   0.066353  0.000123278   0.00135236   0.00351135  0.000204886
  6     5301.3809    0.19111   0.00120494   0.00174642  0.000962847  0.000714764
  7     5296.7947    0.11693  0.000322236  7.55989e-05  0.000161495  0.000299729
  8     5

## Try to turn off generator status at gen 2

In [ ]:
# r1_mpc['gen'] is a numpy array containing generator data.

# The columns are indexed using variables obtained from m.idx_gen().
# For example:

# PG = column for real power output (MW).
# PMAX = column for the maximum generator limit.
# GEN_STATUS = column for status (1 = on, 0 = off).

# The Cons for this case 2 = if we don't check exactly how many generators in case that will be use in simulation we can not guess the gen_index
# So for the Solutions, may you that use this case 2 as a step simulation in your research/experiment check first how many PG and Generator Status that available
# from the use of caseframes available at matpowercaseframes too.
# The Pros : for faster looping simulation it is far more better if use in many hour scenario (maybe, haven't test it yet)

[GEN_BUS, PG, QG, QMAX, QMIN, VG, MBASE, GEN_STATUS, PMAX, PMIN, 
 MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN, PC1, PC2, QC1MIN, QC1MAX, 
 QC2MIN, QC2MAX, RAMP_AGC, RAMP_10, RAMP_30, RAMP_Q, APF] = m.idx_gen(nout='max_nout')

gen_index = 2
gen_index_ = int(gen_index - 1)
gen_index_


1

In [ ]:
GEN_STATUS_ = int(GEN_STATUS - 1) # -1 due to python indexing start from 0
r1_mpc['gen'][gen_index_,GEN_STATUS_] = 0

In [ ]:
#Check the status of generators, generator 2 must be turn off
print(r1_mpc['gen'][:, GEN_STATUS_])

[1. 0. 1.]


In [ ]:
#re-run power flow with modified generator
# push back value to octave client
m.push('mpc', r1_mpc) # push r1_mpc in python to mpc in octave

# test if we can retrive pushed value
mpc = m.pull('mpc')

# test if our pushed variable can be used
m.eval("_r1 = runopf(mpc, mpopt);")


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0          5535                    0.14       0.1765      57.0117            0
  1     7197.6704    0.52932    0.0149403      7.34106      11.3047     0.107027
  2     7373.1882    0.42799   0.00136366     0.755348        2.122    0.0102059
  3     7371.1188   0.021125  1.87368e-05    0.0689365     0.222036  0.000119116
  4     7366.8124   0.043537  3.30514e-05   0.00636179    0.0224058  0.000247908
  5     7360.7088   0.072788  0.000218749  0.000746286   0.00422756  0.000351449
  6      7341.446    0.23039   0.00171759   0.00268994   0.00111216   0.00110956
  7      7335.251    0.11765  0.000371412  0.000212541  0.000179507  0.000357238
  8     7

Recaps:
 1. Case 1 = Used matpowercaseframes, for understand and trying far more better because it is neat and easy to understand code structure to modify (from my point of view),

 1. Case 2 = Direct approach, for simulation looping I guess it is more efficient but not too friendly understand the code structure to modify (from my point of view).

 The points : 
 1. If you really have intention from the first and know what generators location that you want to turn off better use matpowercaseframes (Case 1) then simulate it, under assumptions that you have known what results from base case (*my opinion: use different scripts to run base case and save result to excel*).

 1. But, if you want to see base case results simulation, modify and then rerun modify version to see results simulation better use Case 2, for looping there will be different scripts.